In [1]:
import logging
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

import cnn_utils

In [2]:
# Setup logging.
reload(logging)
logging.basicConfig(format = "[%(asctime)s] %(message)s",
                    datefmt = "%Y-%m-%d %H:%M:%S",
                    level = logging.INFO)

# Create an interactive session.
sess = tf.InteractiveSession()

In [3]:
# Load the dataset.
dataset = mnist_input_data.read_data_sets("../MNIST_data", one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
"""Define the network."""

# Input and output placeholders.
x = tf.placeholder(tf.float32, shape=[None, 784], name = "input")
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Build the network.
keep_prob = tf.placeholder(tf.float32, name = "dropout")
y = cnn_utils.inference(x, keep_prob)
loss = cnn_utils.loss(y, y_)
accuracy = cnn_utils.accuracy(y, y_)

# Setup training.
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = cnn_utils.training(loss, accuracy, optimizer)
saver = tf.train.Saver()

summary = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("checkpoint", sess.graph)

# Save the graph.
tf.train.write_graph(sess.graph_def, "checkpoint", "graph.pb", as_text=True)

'checkpoint/graph.pb'

In [5]:
# Perform the training.
sess.run(tf.global_variables_initializer())
validation_feed_dict = {
    x:dataset.validation.images,
    y_: dataset.validation.labels,
    keep_prob: 1.0
}
for i in range(20000):
    batch = dataset.train.next_batch(50)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if (i+1) % 100 == 0:
        validation_accuracy = accuracy.eval(feed_dict=validation_feed_dict)
        if (i+1) <= 2000 or (i+1) % 1000 == 0:
            # Print less frequently when i gets big to avoid verbose printouts.
            logging.info("step %d, validation accuracy %.2f%%" %
                         (i+1, validation_accuracy * 100))
        saver.save(sess, "checkpoint/cnn", global_step = i+1)
        
        summary_str = sess.run(summary, feed_dict=validation_feed_dict)
        summary_writer.add_summary(summary_str, i+1)
        summary_writer.flush()

[2017-02-15 23:15:27] step 100, validation accuracy 84.46%
[2017-02-15 23:15:29] step 200, validation accuracy 90.84%
[2017-02-15 23:15:30] step 300, validation accuracy 92.50%
[2017-02-15 23:15:32] step 400, validation accuracy 94.22%
[2017-02-15 23:15:34] step 500, validation accuracy 94.64%
[2017-02-15 23:15:35] step 600, validation accuracy 95.48%
[2017-02-15 23:15:37] step 700, validation accuracy 96.16%
[2017-02-15 23:15:38] step 800, validation accuracy 96.14%
[2017-02-15 23:15:40] step 900, validation accuracy 96.24%
[2017-02-15 23:15:41] step 1000, validation accuracy 96.86%
[2017-02-15 23:15:43] step 1100, validation accuracy 96.88%
[2017-02-15 23:15:44] step 1200, validation accuracy 97.02%
[2017-02-15 23:15:46] step 1300, validation accuracy 97.12%
[2017-02-15 23:15:48] step 1400, validation accuracy 97.38%
[2017-02-15 23:15:49] step 1500, validation accuracy 97.38%
[2017-02-15 23:15:51] step 1600, validation accuracy 97.40%
[2017-02-15 23:15:52] step 1700, validation accur

Run `tensorboard --logdir checkpoint/` to visualize training progress.

In [6]:
# Run inference and get accuracy.
print("test accuracy %g" % 
      accuracy.eval(feed_dict={
            x: dataset.test.images, 
            y_: dataset.test.labels, keep_prob: 1.0}))

test accuracy 0.9921


Copy the `checkpoint` folder to `trained-models/cnn`.